In [101]:
import json
import gradio as gr
import requests

In [102]:
system_message = """
You are FlightAI, a travel assistant that can use tools to retrieve information.
If a user asks about flight ticket prices, always use the `get_ticket_price` tool
instead of saying you don't have real-time information.
Respond naturally with the retrieved price.
"""

In [103]:
def chat(message, history):
    history = [{"role":h["role"],"content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = requests.post(
        "http://localhost:11434/api/chat",
        json = {
            "model" : 'llama3.2',
            "messages" : messages,
            "stream" : False
        }
    )
    return response.json()['message']['content']

In [104]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7878
* To create a public link, set `share=True` in `launch()`.


In [105]:
def get_ticket_price(destination_city):
    ticket_prices = {
        "london": 850,
        "new york": 600,
        "los angeles": 720,
        "paris": 780,
        "tokyo": 1200
    }
    if destination_city:
        price = ticket_prices.get(destination_city.lower(), "Unknown")
        return {"city": destination_city, "price": price}
    else:
        return {"error": "No destination city provided"}

In [106]:
get_ticket_price("PARIS")

{'city': 'PARIS', 'price': 780}

In [107]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_ticket_price",
            "description": "Get the current ticket price to a specific city.",
            "parameters": {
                "type": "object",
                "properties": {
                    "destination_city": {
                        "type": "string",
                        "description": "Name of the destination city, e.g. 'London' or 'New York'."
                    }
                },
                "required": ["destination_city"]
            }
        }
    }
]


In [108]:
import json
import requests

def chat(message, history=None):
    if history is None:
        history = []

    # Prepare message list
    messages = [{"role": "system", "content": system_message}] + history + [
        {"role": "user", "content": message}
    ]

    # Step 1: Send message to Ollama
    response = requests.post(
        "http://localhost:11434/api/chat",
        json={
            "model": "llama3.2:1b",
            "messages": messages,
            "tools": tools,
            "stream": False
        }
    )

    data = response.json()

    # Step 2: Handle tool calls if any
    if data.get("message") and "tool_calls" in data["message"]:
        print("Model requested a tool call.")
        message_obj = data["message"]
        tool_response = handle_tool_call(message_obj)

        # ✅ Always wrap tool_response in a list
        if not isinstance(tool_response, list):
            tool_response = [tool_response]

        # Append tool call + tool output properly
        messages.append(message_obj)
        messages.extend(tool_response)

        # Step 3: Send messages again after tool execution
        response = requests.post(
            "http://localhost:11434/api/chat",
            json={
                "model": "llama3.2:1b",
                "messages": messages,
                "stream": False
            }
        )

        data = response.json()
        msg = data.get("message")
        if msg and "content" in msg:
            return msg["content"]

    # Step 4: If no tool call
    if data.get("message", {}).get("content"):
        return data["message"]["content"]

    # Step 5: Fallback
    return "No response from model."


In [109]:
def handle_tool_call(message):
    results = []  # collect tool outputs

    for tool_call in message.get("tool_calls", []):
        fn_name = tool_call.get("function", {}).get("name")
        raw_args = tool_call.get("function", {}).get("arguments", {})

        try:
            args = json.loads(raw_args) if isinstance(raw_args, str) else raw_args
        except json.JSONDecodeError:
            args = {}

        print("DEBUG - Tool call:", fn_name)
        print("DEBUG - Raw args:", raw_args)
        print("DEBUG - Parsed args:", args)

        if fn_name == "get_ticket_price":
            destination_city = args.get("destination_city")
            result = get_ticket_price(destination_city)
        else:
            result = {"error": f"Unknown tool: {fn_name}"}

        results.append({
            "role": "tool",
            "tool_call_id": tool_call.get("id", str(uuid.uuid4())),
            "content": json.dumps(result)
        })

    return results


In [110]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7879
* To create a public link, set `share=True` in `launch()`.


Model requested a tool call.
DEBUG - Tool call: get_ticket_price
DEBUG - Raw args: {'required': '["destination_city"]', 'type': 'string'}
DEBUG - Parsed args: {'required': '["destination_city"]', 'type': 'string'}
Model requested a tool call.
DEBUG - Tool call: get_ticket_price
DEBUG - Raw args: {'destination_city': 'London'}
DEBUG - Parsed args: {'destination_city': 'London'}
Model requested a tool call.
DEBUG - Tool call: get_ticket_price
DEBUG - Raw args: {'destination_city': 'newyork'}
DEBUG - Parsed args: {'destination_city': 'newyork'}
Model requested a tool call.
DEBUG - Tool call: get_ticket_price
DEBUG - Raw args: {'destination_city': 'New York'}
DEBUG - Parsed args: {'destination_city': 'New York'}
Model requested a tool call.
DEBUG - Tool call: get_ticket_price
DEBUG - Raw args: {'properties': {'destination_city': {'description': "Name of the destination city, e.g. 'London' or 'New York'.", 'type': 'string'}}, 'required': ['destination_city'], 'type': 'object'}
DEBUG - Parse